In [1]:
# Import des librairies

import numpy as np 
import pandas as pd 

# Libraries for Recommendation System
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors


In [2]:
# Je met les deux dataset dont j'ai besoin dans des variables pour le filtrage coolaboratif item-item 
data_ratings = pd.read_csv("ratings.csv")
data_books = pd.read_csv("books.csv")

In [3]:
data_ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [4]:
# Dans books on va se servir des colonnes book_id et titre donc je vais créer un dataframme contenant ces deux colonnes


ratings = data_ratings.loc[:,{"user_id","book_id","rating"}]
books = data_books.loc[:,{"book_id","title"}]

In [5]:
ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [6]:
len(books["title"].unique())

9964

In [7]:
data = pd.merge(books,ratings)
user_book_table = data.pivot_table(index = ["title"],columns = ["user_id"],values = "rating").fillna(0)
user_book_table

user_id,1,2,3,4,5,6,7,8,9,10,...,53415,53416,53417,53418,53419,53420,53421,53422,53423,53424
title,,,,,,,,,,,,,,,,,,,,,
"Angels (Walsh Family, #3)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#GIRLBOSS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
"'Tis (Frank McCourt, #2)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
واحة الغروب,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
يوتوبيا,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ڤيرتيجو,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
query_index = np.random.choice(user_book_table.shape[0])
print("Choosen Book is: ",user_book_table.index[query_index])

Choosen Book is:  The Tender Bar


In [9]:
user_book_table_matrix = csr_matrix(user_book_table.values)
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(user_book_table_matrix)
distances, indices = model_knn.kneighbors(user_book_table.iloc[query_index,:].values.reshape(1,-1), n_neighbors = 6)

In [10]:
book = []
distance = []

for i in range(0, len(distances.flatten())):
    if i != 0:
        book.append(user_book_table.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])    

b=pd.Series(book,name='book')
d=pd.Series(distance,name='distance')
recommend = pd.concat([b,d], axis=1)
recommend = recommend.sort_values('distance',ascending=False)

print('Recommendations for {0}:\n'.format(user_book_table.index[query_index]))
for i in range(0,recommend.shape[0]):
    print('{0}: {1}, with distance of {2}'.format(i, recommend["book"].iloc[i], recommend["distance"].iloc[i]))

Recommendations for The Tender Bar:

0: Don't Let's Go to the Dogs Tonight, with distance of 0.9009158035060412
1: Truth and Beauty, with distance of 0.8987941781295024
2: Angela's Ashes (Frank McCourt, #1), with distance of 0.8986723868901239
3: The Glass Castle, with distance of 0.8884414349074389
4: The Liars' Club, with distance of 0.8430769436942744


In [11]:
import joblib 
joblib.dump(model_knn,"KNN_model.sav" )

['KNN_model.sav']

In [14]:
type(distances)


numpy.ndarray

In [13]:
type(indices)

numpy.ndarray